Copyright (c) Microsoft Corporation. All rights reserved.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/regression-part2-automated-ml.png)

# Tutorial: Use automated machine learning to predict taxi fares

In this tutorial, you use automated machine learning in Azure Machine Learning service to create a regression model to predict NYC taxi fare prices. This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

In this tutorial you learn the following tasks:

* Download, transform, and clean data using Azure Open Datasets
* Train an automated machine learning regression model
* Calculate model accuracy

If you donâ€™t have an Azure subscription, create a free account before you begin. Try the [free or paid version](https://aka.ms/AMLFree) of Azure Machine Learning service today.

## Prerequisites

* Complete the [setup tutorial](https://docs.microsoft.com/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup) if you don't already have an Azure Machine Learning service workspace or notebook virtual machine.
* After you complete the setup tutorial, open the **tutorials/regression-automated-ml.ipynb** notebook using the same notebook server.

This tutorial is also available on [GitHub](https://github.com/Azure/MachineLearningNotebooks/tree/master/tutorials) if you wish to run it in your own [local environment](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-environment#local). Run `pip install azureml-sdk[automl] azureml-opendatasets azureml-widgets` to get the required packages.

## Download and prepare data

Import the necessary packages. The Open Datasets package contains a class representing each data source (`NycTlcGreen` for example) to easily filter date parameters before downloading.

In [2]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

Begin by creating a dataframe to hold the taxi data. When working in a non-Spark environment, Open Datasets only allows downloading one month of data at a time with certain classes to avoid `MemoryError` with large datasets. To download taxi data, iteratively fetch one month at a time, and before appending it to `green_taxi_df` randomly sample 2,000 records from each month to avoid bloating the dataframe. Then preview the data.

In [3]:
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))
    
green_taxi_df.head(10)

ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=1/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=1/part-00175-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2745-1.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=34836.43 [ms]
ActivityCompleted: Activity=to_pandas_dataframe, HowEnded=Success, Duration=34854.41 [ms]
ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=2/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=2/part-00007-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2577-1.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=26389.23 [

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
1058780,2,2015-01-30 21:45:12,2015-01-30 22:05:31,1,4.34,None,None,-73.977142,40.687443,-73.957779,...,1,17.0,0.5,0.5,0.3,3.66,0.0,NaN,21.96,1.0
419965,2,2015-01-19 22:52:45,2015-01-19 23:04:58,1,3.39,None,None,-73.948463,40.718418,-73.900253,...,2,12.0,0.5,0.5,0.3,0.00,0.0,NaN,13.30,1.0
662616,1,2015-01-09 22:26:23,2015-01-09 22:41:13,1,3.30,None,None,-73.912910,40.744251,-73.953888,...,2,13.5,0.5,0.5,0.3,0.00,0.0,NaN,14.80,1.0
332263,2,2015-01-15 13:38:51,2015-01-15 13:39:04,1,0.00,None,None,-73.902016,40.763786,-73.902100,...,2,2.5,0.0,0.5,0.3,0.00,0.0,NaN,3.30,1.0
1287329,1,2015-01-14 17:04:52,2015-01-14 17:10:10,1,1.00,None,None,-73.947151,40.805775,-73.946495,...,2,6.0,0.5,0.5,0.3,0.00,0.0,NaN,7.30,1.0
1119254,2,2015-01-17 19:33:29,2015-01-17 19:47:34,1,3.11,None,None,-73.956871,40.712681,-73.968674,...,1,13.0,0.0,0.5,0.3,2.60,0.0,NaN,16.40,1.0
1232638,2,2015-01-28 18:35:00,2015-01-28 18:45:42,1,5.55,None,None,-73.844818,40.720062,-73.872993,...,1,16.5,1.0,0.5,0.3,3.66,0.0,NaN,21.96,1.0
769920,2,2015-01-17 16:28:20,2015-01-17 16:46:37,1,2.50,None,None,-73.918999,40.742996,-73.905029,...,2,12.0,0.0,0.5,0.3,0.00,0.0,NaN,12.80,1.0
74342,1,2015-01-10 15:48:12,2015-01-10 15:53:09,1,0.70,None,None,-73.964615,40.683086,-73.965103,...,1,5.0,0.0,0.5,0.3,0.00,0.0,NaN,5.80,1.0
725451,1,2015-01-15 23:29:51,2015-01-15 23:50:59,1,6.00,None,None,-73.998405,40.682285,-73.934998,...,1,20.5,0.5,0.5,0.3,5.00,0.0,NaN,26.80,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day, and will allow the model to factor in time-based seasonality. 

Use the `apply()` function on the dataframe to iteratively apply the `build_time_features()` function to each row in the taxi data.

In [4]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,month_num,day_of_month,day_of_week,hour_of_day
1058780,2,2015-01-30 21:45:12,2015-01-30 22:05:31,1,4.34,None,None,-73.977142,40.687443,-73.957779,...,0.3,3.66,0.0,NaN,21.96,1.0,1,30,4,21
419965,2,2015-01-19 22:52:45,2015-01-19 23:04:58,1,3.39,None,None,-73.948463,40.718418,-73.900253,...,0.3,0.00,0.0,NaN,13.30,1.0,1,19,0,22
662616,1,2015-01-09 22:26:23,2015-01-09 22:41:13,1,3.30,None,None,-73.912910,40.744251,-73.953888,...,0.3,0.00,0.0,NaN,14.80,1.0,1,9,4,22
332263,2,2015-01-15 13:38:51,2015-01-15 13:39:04,1,0.00,None,None,-73.902016,40.763786,-73.902100,...,0.3,0.00,0.0,NaN,3.30,1.0,1,15,3,13
1287329,1,2015-01-14 17:04:52,2015-01-14 17:10:10,1,1.00,None,None,-73.947151,40.805775,-73.946495,...,0.3,0.00,0.0,NaN,7.30,1.0,1,14,2,17
1119254,2,2015-01-17 19:33:29,2015-01-17 19:47:34,1,3.11,None,None,-73.956871,40.712681,-73.968674,...,0.3,2.60,0.0,NaN,16.40,1.0,1,17,5,19
1232638,2,2015-01-28 18:35:00,2015-01-28 18:45:42,1,5.55,None,None,-73.844818,40.720062,-73.872993,...,0.3,3.66,0.0,NaN,21.96,1.0,1,28,2,18
769920,2,2015-01-17 16:28:20,2015-01-17 16:46:37,1,2.50,None,None,-73.918999,40.742996,-73.905029,...,0.3,0.00,0.0,NaN,12.80,1.0,1,17,5,16
74342,1,2015-01-10 15:48:12,2015-01-10 15:53:09,1,0.70,None,None,-73.964615,40.683086,-73.965103,...,0.3,0.00,0.0,NaN,5.80,1.0,1,10,5,15
725451,1,2015-01-15 23:29:51,2015-01-15 23:50:59,1,6.00,None,None,-73.998405,40.682285,-73.934998,...,0.3,5.00,0.0,NaN,26.80,1.0,1,15,3,23


Remove some of the columns that you won't need for training or additional feature building.

In [5]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)
    
green_taxi_df.head(5)

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
1058780,2,1,4.34,-73.977142,40.687443,-73.957779,40.638474,21.96,1,30,4,21
419965,2,1,3.39,-73.948463,40.718418,-73.900253,40.700733,13.30,1,19,0,22
662616,1,1,3.30,-73.912910,40.744251,-73.953888,40.719372,14.80,1,9,4,22
332263,2,1,0.00,-73.902016,40.763786,-73.902100,40.763802,3.30,1,15,3,13
1287329,1,1,1.00,-73.947151,40.805775,-73.946495,40.795265,7.30,1,14,2,17


### Cleanse data 

Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [6]:
green_taxi_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.786792,1.366375,2.891429,-73.839281,40.696712,-73.813677,40.681061,14.826679,6.500000,15.119333,3.279792,13.587375
std,0.409582,1.038600,2.929296,2.655901,1.464727,2.978464,1.642306,11.859068,3.452124,8.462116,1.957337,6.788682
min,1.000000,0.000000,0.000000,-74.337700,0.000000,-74.337708,0.000000,-100.000000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.080000,-73.959473,40.699279,-73.967188,40.700104,8.000000,3.750000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.930000,-73.944923,40.746868,-73.944042,40.747963,11.300000,6.500000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.650000,-73.916321,40.803295,-73.909163,40.790906,17.800000,9.250000,22.000000,5.000000,19.000000
max,2.000000,8.000000,41.620000,0.000000,40.967804,0.000000,41.126099,400.000000,12.000000,30.000000,6.000000,23.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the bounds of the Manhattan area. This will filter out longer taxi trips or trips that are outliers in respect to their relationship with other features. 

Additionally filter the `tripDistance` field to be greater than zero but less than 31 miles (the haversine distance between the two lat/long pairs). This eliminates long outlier trips that have inconsistent trip cost.

Lastly, the `totalAmount` field has negative values for the taxi fares, which don't make sense in the context of our model, and the `passengerCount` field has bad data with the minimum values being zero.

Filter out these anomalies using query functions, and then remove the last few columns unnecessary for training.

In [7]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

Call `describe()` again on the data to ensure cleansing worked as expected. You now have a prepared and cleansed set of taxi, holiday, and weather data to use for machine learning model training.

In [8]:
final_df.describe()

,vendorID,passengerCount,tripDistance,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,23274.000000,23274.000000,23274.000000,23274.00000,23274.000000,23274.000000,23274.000000,23274.000000
mean,1.787359,1.368995,2.951363,14.83614,6.508207,15.111799,3.286199,13.607459
std,0.409184,1.043526,2.882996,10.64191,3.454369,8.461324,1.954721,6.794824
min,1.000000,1.000000,0.250000,0.07000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.110000,8.30000,4.000000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,2.000000,11.62000,7.000000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.700000,17.80000,10.000000,22.000000,5.000000,19.000000
max,2.000000,8.000000,30.100000,283.75000,12.000000,30.000000,6.000000,23.000000


## Configure workspace


Create a workspace object from the existing workspace. A [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is a class that accepts your Azure subscription and resource information. It also creates a cloud resource to monitor and track your model runs. `Workspace.from_config()` reads the file **config.json** and loads the authentication details into an object named `ws`. `ws` is used throughout the rest of the code in this tutorial.

In [9]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

## Split the data into train and test sets

Split the data into training and test sets by using the `train_test_split` function in the `scikit-learn` library. This function segregates the data into the x (**features**) data set for model training and the y (**values to predict**) data set for testing. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random generator, so that your train-test splits are deterministic.

In [10]:
from sklearn.model_selection import train_test_split

y_df = final_df.pop("totalAmount")
x_df = final_df

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

The purpose of this step is to have data points to test the finished model that haven't been used to train the model, in order to measure true accuracy. 

In other words, a well-trained model should be able to accurately make predictions from data it hasn't already seen. You now have data prepared for auto-training a machine learning model.

## Automatically train a model

To automatically train a model, take the following steps:
1. Define settings for the experiment run. Attach your training data to the configuration, and modify settings that control the training process.
1. Submit the experiment for model tuning. After submitting the experiment, the process iterates through different machine learning algorithms and hyperparameter settings, adhering to your defined constraints. It chooses the best-fit model by optimizing an accuracy metric.

### Define training settings

Define the experiment parameter and model settings for training. View the full list of [settings](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-auto-train). Submitting the experiment with these default settings will take approximately 5-10 min, but if you want a shorter run time, reduce the `iterations` parameter.


|Property| Value in this tutorial |Description|
|----|----|---|
|**iteration_timeout_minutes**|2|Time limit in minutes for each iteration. Reduce this value to decrease total runtime.|
|**iterations**|20|Number of iterations. In each iteration, a new machine learning model is trained with your data. This is the primary value that affects total run time.|
|**primary_metric**| spearman_correlation | Metric that you want to optimize. The best-fit model will be chosen based on this metric.|
|**preprocess**| True | By using **True**, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)|
|**verbosity**| logging.INFO | Controls the level of logging.|
|**n_cross_validations**|5|Number of cross-validation splits to perform when validation data is not specified.|

In [11]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "iterations": 20,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

Use your defined training settings as a `**kwargs` parameter to an `AutoMLConfig` object. Additionally, specify your training data and the type of model, which is `regression` in this case.

In [12]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             X=x_train.values,
                             y=y_train.values.flatten(),
                             **automl_settings)

Automated machine learning pre-processing steps (feature normalization, handling missing data, converting text to numeric, etc.) become part of the underlying model. When using the model for predictions, the same pre-processing steps applied during training are applied to your input data automatically.

### Train the automatic regression model

Create an experiment object in your workspace. An experiment acts as a container for your individual runs. Pass the defined `automl_config` object to the experiment, and set the output to `True` to view progress during the run. 

After starting the experiment, the output shown updates live as the experiment runs. For each iteration, you see the model type, the run duration, and the training accuracy. The field `BEST` tracks the best running training score based on your metric type.

In [13]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_39053d4c-0a34-4597-8afc-6f9485ef21c0
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High Cardinality Feature Detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.

**************************************************************

## Explore the results

Explore the results of automatic training with a [Jupyter widget](https://docs.microsoft.com/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py). The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.

In [14]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Retrieve the best model

Select the best model from your iterations. The `get_output` function returns the best run and the fitted model for the last fit invocation. By using the overloads on `get_output`, you can retrieve the best run and fitted model for any logged metric or a particular iteration.

In [15]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: taxi-experiment,
Id: AutoML_39053d4c-0a34-4597-8afc-6f9485ef21c0_18,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('pref...666666666667, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.13333333333333333]))]),
          stddev=None)


### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function `predict` uses the best model and predicts the values of y, **trip cost**, from the `x_test` data set. Print the first 10 predicted cost values from `y_predict`.

In [16]:
y_predict = fitted_model.predict(x_test.values)
print(y_predict[:10])

[ 6.96754769 24.0207693  10.84190221 17.65025257 20.64950238 10.84643437
  8.51986589  5.98507761  9.96591235 14.02266534]


Calculate the `root mean squared error` of the results. Convert the `y_test` dataframe to a list to compare to the predicted values. The function `mean_squared_error` takes two arrays of values and calculates the average squared error between them. Taking the square root of the result gives an error in the same units as the y variable, **cost**. It indicates roughly how far the taxi fare predictions are from the actual fares.

In [17]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

5.114012828936199

Run the following code to calculate mean absolute percent error (MAPE) by using the full `y_actual` and `y_predict` data sets. This metric calculates an absolute difference between each predicted and actual value and sums all the differences. Then it expresses that sum as a percent of the total of the actual values.

In [18]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.14109471367729093

Model Accuracy:
0.8589052863227091


From the two prediction accuracy metrics, you see that the model is fairly good at predicting taxi fares from the data set's features, typically within +- $4.00, and approximately 15% error. 

The traditional machine learning model development process is highly resource-intensive, and requires significant domain knowledge and time investment to run and compare the results of dozens of models. Using automated machine learning is a great way to rapidly test many different models for your scenario.

## Clean up resources

Do not complete this section if you plan on running other Azure Machine Learning service tutorials.

### Stop the notebook VM

If you used a cloud notebook server, stop the VM when you are not using it to reduce cost.

1. In your workspace, select **Notebook VMs**.
1. From the list, select the VM.
1. Select **Stop**.
1. When you're ready to use the server again, select **Start**.

### Delete everything

If you don't plan to use the resources you created, delete them, so you don't incur any charges.

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

## Next steps

In this automated machine learning tutorial, you did the following tasks:

> * Configured a workspace and prepared data for an experiment.
> * Trained by using an automated regression model locally with custom parameters.
> * Explored and reviewed training results.

[Deploy your model](https://docs.microsoft.com/azure/machine-learning/service/tutorial-deploy-models-with-aml) with Azure Machine Learning service.